# Preliminary

This is the annotated version of the Universal Transformer paper utilizing graph-based neural networks and attention mechanisms, to be used for educational purposes. The text was transcribed verbatim. Besides this preliminary section, **I did not write the main text and do not claim to have written this paper**. My comments and equations are my own and annotated by block quotes.

# The Annotated Universal [Graph] Transformer

> The following non-block quote text is copied verbatim from the [Universal Transformers](https://arxiv.org/pdf/1807.03819.pdf) paper. Code and block quotes are my own. Spacing and paragraphs inserted when appropriate to break up text and code cells.

## Introduction

Convolutional and fully-attentional feed-forward architectures like the Transformer have recentlyemerged  as  viable  alternatives  to  recurrent  neural  networks  (RNNs)  for  a  range  of  sequencemodeling tasks, notably machine translation (Gehring et al., 2017; Vaswani et al., 2017).  Theseparallel-in-time architectures address a significant shortcoming of RNNs, namely their inherentlysequential computation which prevents parallelization across elements of the input sequence, whilststill addressing the vanishing gradients problem as the sequence length gets longer (Hochreiter et al.,2003). The Transformer model in particular relies entirely on a self-attention mechanism (Parikh et al.,2016; Lin et al., 2017) to compute a series of context-informed vector-space representations of thesymbols in its input and output, which are then used to predict distributions over subsequent symbols asthe model predicts the output sequence symbol-by-symbol. Not only is this mechanism straightforwardto parallelize, but as each symbol’s representation is also directly informed by all other symbols’representations, this results in an effectively global receptive field across the whole sequence. This stands in contrast to e.g. convolutional architectures which typically only have a limited receptive field.

Notably, however, the Transformer with its fixed stack of distinct layers foregoes RNNs’ inductive biastowards learning iterative or recursive transformations. Our experiments indicate that this inductive bias may be crucial for several algorithmic and language understanding tasks of varying complexity:in contrast to models such as the Neural Turing Machine (Graves et al., 2014), the Neural GPU (Kaiser& Sutskever, 2016) or Stack RNNs (Joulin & Mikolov, 2015), the Transformer does not generalizewell to input lengths not encountered during training.

In this paper, we introduce theUniversal Transformer (UT), a parallel-in-time recurrent self-attentivesequence model which can be cast as a generalization of the Transformer model, yielding increasedtheoretical capabilities and improved results on a wide range of challenging sequence-to-sequencetasks. UTs combine the parallelizability and global receptive field of feed-forward sequence modelslike the Transformer with the recurrent inductive bias of RNNs, which seems to be better suited toa range of algorithmic and natural language understanding sequence-to-sequence problems. As thename implies, and in contrast to the standard Transformer, under certain assumptions UTs can beshown to be Turing-complete (or “computationally universal”, as shown in Section 4).

In each recurrent step, the Universal Transformer iteratively refines its representations for all symbolsin the sequence in parallel using a self-attention mechanism (Parikh et al., 2016; Lin et al., 2017),followed by a transformation (shared across all positions and time-steps) consisting of a depth-wiseseparable convolution (Chollet, 2016; Kaiser et al., 2017) or a position-wise fully-connected layer(see Fig 1). We also add a dynamic per-position halting mechanism (Graves, 2016), allowing the modelto choose the required number of refinement stepsfor each symboldynamically, and show for the firsttime that such a conditional computation mechanism can in fact improve accuracy on several smaller,structured algorithmic and linguistic inference tasks (although it marginally degraded results on MT).

Our strong experimental results show that UTs outperform Transformers and LSTMs across a widerange of tasks.  The added recurrence yields improved results in machine translation where UTsoutperform the standard Transformer.  In experiments on several algorithmic tasks and the bAbIlanguage understanding task, UTs also consistently and significantly improve over LSTMs and thestandard Transformer. Furthermore, on the challenging LAMBADA text understanding data set UTswith dynamic halting achieve a new state of the art.

## Model Description

### The Universal Transformer

The Universal Transformer (UT; see Fig. 2) is based on the popular encoder-decoder architecturecommonly used in most neural sequence-to-sequence models (Sutskever et al., 2014; Cho et al., 2014;Vaswani et al., 2017). Both the encoder and decoder of the UT operate by applying a recurrent neuralnetwork to the representations of each of the positions of the input and output sequence, respectively.However, in contrast to most applications of recurrent neural networks to sequential data, the UT doesnot recur over positions in the sequence, but over consecutive revisions of the vector representations ofeach position (i.e., over “depth”). In other words, the UT is not computationally bound by the numberof symbols in the sequence, but only by the number of revisions made to each symbol’s representation.

n each recurrent time-step, the representation of every position is concurrently (in parallel) revisedin two sub-steps: first, using a self-attention mechanism to exchange information across all positionsin the sequence, thereby generating a vector representation for each position that is informed by therepresentations of all other positions at the previous time-step. Then, by applying a transition function(shared across position and time) to the outputs of the self-attention mechanism, independently ateach position. As the recurrent transition function can be applied any number of times, this impliesthat UTs can have variable depth (number of per-symbol processing steps). Crucially, this is in contrastto most popular neural sequence models, including the Transformer (Vaswani et al., 2017) or deepRNNs, which have constant depth as a result of applying afixed stackof layers.  We now describethe encoder and decoder in more detail.

#### ENCODER

Given an input sequence of lengthm, we start with a matrix whose rows are initializedas the $d$-dimensional embeddings of the symbols at each position of the sequence $H^0 \in \mathbb{R}^{m \times d}$. The UT then iteratively computes representations $H^t$ at step $t$ for all $m$ positions in parallel by applyingthe multi-headed dot-product self-attention mechanism from Vaswani et al. (2017), followed by arecurrent transition function. We also add residual connections around each of these function blocksand apply dropout and layer normalization (Srivastava et al., 2014; Ba et al., 2016) (see Fig. 2 for asimplified diagram, and Fig. 4 in the Appendix A for the complete model.).

More specifically, we use the scaled dot-product attention which combines queries $Q$, keys $K$, and values $V$ as follows

$$
\text{Attention}(Q,K,V) = \text{SOFTMAX} \Bigg( \frac{Q K^\top}{\sqrt{d}} \Bigg) V
$$

where $d$ is the number of columns of $Q$, $K$, and $V$. 

> The **attention** layer, each node in module learns to assign weights on its incoming edges. For node pair $(i,j)$ with node $x_i,x_j \in \mathbb{R}^n$, the score of connection is as follows:
$$
q_j = W_q \cdot x_j \\
k_i = W_k \cdot x_i \\
v_i = W_v \cdot x_i \\
\text{score} = q_j k_i
$$
$W_q, W_k, W_v \in \mathbb{R}^{n \times d_k}$ map the representations of x to "query", "key", and "value" space repsectively. These values are three different linear projections of the data. For the "query" case ($W_j$), these are linear projections of source nodes for edges. For "key" and "value", were are linear projections of the destination nodes. The dot product between query source nodes and key destination nodes computes the score of the given connection. See [attention_explained.ipynb](./attention_explained.ipynb) for further information.


We use the multi-head version with $k$ heads, as introduced in Vaswani et al., 2017.

$$
\text{MultiHeadSelfAttention}(H^T) = \text{Concat}(\text{head}_1,\dots,\text{head}_k) W^O \\
\text{where } \text{head}_i = \text{Attention}(H^t W_i^Q, H^t W_i^K, H^t W_i^V)
$$

and we map the state $H^t$ to queries, key, and values with affine projections using learned parameter matricies $W^Q \in \mathbb{R}^{d \times d / k}$, $W^K \in \mathbb{R}^{d \times d \ k}$, $W^V \in \mathbb{R}^{d \times d \ k}$, and $W^O \in \mathbb{R}^{d \times d}$

> For **multi-head attention**, we compute a portion of the attention and concatenate the results. Hence attention calculations can occur in parallel. To do this in a tensor-framework, this means we create a multi-dimensional tensor with $\text{heads}$ being one of the dimensions. Lets again walk through this step-wise to get a better understanding of where the **multi-head** part comes in. See [attention_explained.ipynb](./attention_explained.ipynb) for further information.

#### DECODER

> ok $x$
Ok

ok

### Dynamic Halting

## Experiment and Analysis

### bAbi Question-Answering

### Subject-Verb Agreement

### LAMBADA Language Modeling

### Algorithmic Tasks

### Learning to Execute (LTE)

### Machine Translation

## Discussion

## Conclusion

# References